In [ ]:
import pandas as pd
import numpy as np
import sqlite3
import matplotlib.pyplot as plt




In [ ]:
results = pd.read_csv('results.csv')
shootouts = pd.read_csv('shootouts.csv')
goalscorers = pd.read_csv('goalscorers.csv')

